# Finetuning Question Answering on Bigbird


**Fine_tuning Bigbird Extracitve Question Answering in PyTorch**

In [105]:
import torch
torch.cuda.empty_cache

<function torch.cuda.memory.empty_cache() -> None>

Install transformers Library

In [113]:
!pip install -q transformers datasets

In [114]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00


Import libraries

In [119]:
import numpy as np
import pandas as pd

import torch
from torch.optim import Adam
from transformers import BigBirdForQuestionAnswering, BigBirdTokenizerFast


### Load the dataset

In [108]:
from datasets import load_dataset

### Load and split dataset, using small datasets for the sake of model training

In [109]:
train_data, valid_data = load_dataset('squad_v2', split='train[:1%]'), load_dataset('squad_v2', split='validation[:3%]')

In [110]:
train_data[0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

### Getting correct answer text alignment and tokenizing the dataset

In [111]:
# Dataset cleaning and tokenization
# BertTokenizerFast because python tokenizer do not have char_to_token functionality

def correct_alignment(context, answer):

    """ Description: This functions corrects the alignment of answers in the squad dataset that are sometimes off by one or 2 values also adds end_postion index.
    
    inputs: list of contexts and answers
    outputs: Updated list that contains answer_end positions """
    
    start_text = answer['text'][0]
    start_idx = answer['answer_start'][0]
    end_idx = start_idx + len(start_text)

    # When alignment is okay
    if context[start_idx:end_idx] == start_text:
      return start_idx, end_idx    
      # When alignment is off by 1 character
    elif context[start_idx-1:end_idx-1] == start_text:
      return start_idx-1, end_idx-1  
      # when alignment is off by 2 characters
    elif context[start_idx-2:end_idx-2] == start_text:
      return start_idx-2, end_idx-2
    else:
      raise ValueError()

### Tokenize our training dataset

In [131]:
model = BigBirdForQuestionAnswering.from_pretrained("rubentito/bigbird-base-itc-mpdocvqa")
tokenizer = BigBirdTokenizerFast.from_pretrained("rubentito/bigbird-base-itc-mpdocvqa")


## Asking a Question

In [121]:
random_num = np.random.randint(0,len(train_data))
question = train_data["question"][random_num]
text = train_data["context"][random_num]

## Let’s see how many tokens this question and text pair have.

In [122]:
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 318 tokens.


## To look at what our tokenizer is doing, let’s just print out the tokens and their IDs.

In [123]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]         65
▁What      1,968
▁term      3,482
▁describes   8,578
▁the         363
▁qualities  14,583
▁of          387
▁the         363
▁relationship   2,877
▁between   1,123
▁Fr        1,406
é            266
d            168
é            266
ric        1,274
▁and         391
▁Lis      28,647
z            190
t            184
?            131
[SEP]         66
▁Although   5,001
▁the         363
▁two         835
▁displayed   9,167
▁great     1,150
▁respect   2,562
▁and         391
▁admiration  28,607
▁for         430
▁each      1,224
▁other       685
,            112
▁their       612
▁friendship  14,839
▁was         474
▁uneasy   34,745
▁and         391
▁had         651
▁some        718
▁qualities  14,583
▁of          387
▁a           358
▁love      1,943
-            113
hate      37,136
▁relationship   2,877
.            114
▁Harold   24,088
▁C           428
.            114
▁Sc        1,547
hon       24,231
berg       4,001
▁believes   5,905
▁that        427
▁Cho      10,132
pin  

In [132]:
#first occurence of [SEP] token
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
#number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)
#number of tokens in segment B (text)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)
#creating the segment ids
segment_ids = [0]*num_seg_a + [1]*num_seg_b
#making sure that every input token has a segment id
assert len(segment_ids) == len(input_ids)

SEP token index:  20
Number of tokens in segment A:  21
Number of tokens in segment B:  297


## Let’s now feed this to our model.

In [125]:
#token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
output = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))

Attention type 'block_sparse' is not possible if sequence_length: 318 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In [126]:
#tokens with highest start and end scores
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")
    
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))


Question:
What term describes the qualities of the relationship between frédéric and liszt?

Answer:
▁love - hate ▁relationship ..


In [127]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [128]:
model.to(device)

BigBirdForQuestionAnswering(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(50358, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(16, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

## creating a function to perform Question Answer analysis

In [129]:
def sample(sample_question, sample_context):
    input_ids = tokenizer.encode(sample_question, sample_context, truncation=True, max_length=512, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    start_token = input_ids[0][start_index].item()
    end_token = input_ids[0][end_index].item()

    answer = tokenizer.decode(input_ids[0][start_index:end_index+1])

    return answer

In [130]:

sample_question = "For what network, did Beyonce land a major movie role in?"
sample_context = "The remaining band members recorded \"Independent Women Part I\", which appeared on the soundtrack to the 2000 film, Charlie's Angels. It became their best-charting single, topping the U.S. Billboard Hot 100 chart for eleven consecutive weeks. In early 2001, while Destiny's Child was completing their third album, Beyoncé landed a major role in the MTV made-for-television film, Carmen: A Hip Hopera, starring alongside American actor Mekhi Phifer. Set in Philadelphia, the film is a modern interpretation of the 19th century opera Carmen by French composer Georges Bizet. When the third album Survivor was released in May 2001, Luckett and Roberson filed a lawsuit claiming that the songs were aimed at them. The album debuted at number one on the U.S. Billboard 200, with first-week sales of 663,000 copies sold. The album spawned other number-one hits, \"Bootylicious\ and the title track, \"Survivor\", the latter of which earned the group a Grammy Award for Best R&B Performance by a Duo or Group with Vocals. After releasing their holiday album 8 Days of Christmas in October 2001, the group announced a hiatus to further pursue solo careers."

answer = sample(sample_question, sample_context)
print("Sample Question:", sample_question)
print("Sample Context:", sample_context)
print("Answer:", answer)






Sample Question: For what network, did Beyonce land a major movie role in?
Sample Context: The remaining band members recorded "Independent Women Part I", which appeared on the soundtrack to the 2000 film, Charlie's Angels. It became their best-charting single, topping the U.S. Billboard Hot 100 chart for eleven consecutive weeks. In early 2001, while Destiny's Child was completing their third album, Beyoncé landed a major role in the MTV made-for-television film, Carmen: A Hip Hopera, starring alongside American actor Mekhi Phifer. Set in Philadelphia, the film is a modern interpretation of the 19th century opera Carmen by French composer Georges Bizet. When the third album Survivor was released in May 2001, Luckett and Roberson filed a lawsuit claiming that the songs were aimed at them. The album debuted at number one on the U.S. Billboard 200, with first-week sales of 663,000 copies sold. The album spawned other number-one hits, "Bootylicious\ and the title track, "Survivor", the la